In [1]:
!pip install html_table_extractor

In [2]:
import pandas as pd
import numpy as np
import os
import random
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import time
from html_table_extractor.extractor import Extractor

In [3]:
def get_html(url):    
    try:
        ### Get the response for the given input url
        response=requests.get(url)
        soup=BeautifulSoup(response.content)   ### Get the soup object for the input url
        return soup
        
    #### Return the url and error for logs
    except Exception as e:
        return ''

In [4]:
    #### Iterating through all the rows in the table

def table_extraction_naics(soup,class_name):
    #'companyDetail topCompanyDetail'
    ### Source table data extraction
    #data_df = table_extraction(soup,types = 'default',class_name='companyDetail topCompanyDetail')  ### Tables extracted from the given url
    ### Iterate until there are br tags    
    # print('Total Number of breaks:', len(soup_.findAll('br')))
    # while len(soup_.findAll('br')):
    #     soup_.br.replace_with("\n")  ### Replaces the line break tag with new line character

        table = soup.find('table', {'class': class_name})    ### Get all the tables for the given class from a soup object
        table_data = []   ### For storing the row level text data for the entire table 
        anchor_data = []  ### For storing the row level link data for the entire table 
        table_headers=[]
        if table!=None:
            table_headers = [header.getText().replace(':','').strip() for header in table.findAll('th')]    #### Get the table headers from the given table    
            rows = table.findAll('tr')     ### Get all the row level data from table

            table_data = []   ### For storing the row level text data for the entire table 
            anchor_data = []  ### For storing the row level link data for the entire table 
            ids = []          ### For storing index where the links are identified 

            for tr in rows:

                cols = tr.findAll('td')  #### Get the various columns or data present in each row

                table_rows = []   ### Store the content/text of the table 
                anchor_tag = []   ### Store the link/anchor tags of the table

                ### Iterate through the columns for each row to get the text and anchor tags
                for idx,col in enumerate(cols):
                    table_rows.append(col.getText())    ### Get the text level data present in each row and each column

                    if col.findAll('a'):               ### Check if the anchor or link information is present in the html
                        anchor_tag.append(col.find('a').get('href'))     ### If present then get the href link
                        ids.append(idx)                ### Get the indices where the links are found

                if len(table_rows)>0:
                    table_data.append(table_rows)    #### Append the row level content/text to master list

                if len(anchor_tag)>0:
                    anchor_data.append(anchor_tag)   #### Append the row level links/href to master list
        return(table_data,anchor_data,table_headers)

In [5]:
df_naics=pd.read_excel("/content/Naics_with_desc (1).xlsx",sheet_name='Company List')

In [ ]:
df_naics

,Company_Name,City,State,Link,naics_6digit_1_code
0,Scruggs Farm & Supplies LLC,Saltillo,MS,https://www.naics.com/company-profile-page/?co...,111110
1,Silver Lk Httrian Brethren Inc,Clark,SD,https://www.naics.com/company-profile-page/?co...,111110
2,Delta and Pine Land Company,Scott,MS,https://www.naics.com/company-profile-page/?co...,111110
3,Midwest Oilseeds Inc,Adel,IA,https://www.naics.com/company-profile-page/?co...,111110
4,Solae Holdings LLC,Saint Louis,MO,https://www.naics.com/company-profile-page/?co...,111110
...,...,...,...,...,...
9365,Security/Office of Diplomatic,Washington,DC,https://www.naics.com/company-profile-page/?co...,928120
9366,Embassy of Ukraine,Washington,DC,https://www.naics.com/company-profile-page/?co...,928120
9367,Interntional Boundary Wtr Comm,El Paso,TX,https://www.naics.com/company-profile-page/?co...,928120
9368,Department Ecnmic Scial Affirs,New York,NY,https://www.naics.com/company-profile-page/?co...,928120


In [ ]:
output_df = pd.DataFrame()

In [6]:
from tqdm import tqdm

final_output = []


for index,row in tqdm(df_naics[:100].iterrows()):
    output_record = {}
    output_record['Company_Name'] = row['Company_Name']
    output_record['City'] = row['City']
    output_record['State'] = row['State']
    output_record['naics_6digit_1_code'] = row['naics_6digit_1_code']
    output_record['Link'] = row['Link']

    html_link = row['Link']
    print(html_link)
    soup = get_html(html_link)    ### Get the soup object for the given url
    # print(soup) 
    if soup.find('div', {'class': "entry-content"})!=None:
        desc = str(soup.find('div', {'class': "gca-column one-half first"}))
        # print(desc)
        temp=table_extraction_naics(soup,"companyDetail topCompanyDetail")
        company_details  = temp[0]
        reference_urls = temp[1][1:]
        company_url = temp[1][0][0]

        output_record['company_url'] = company_url
        output_record['reference_urls'] = reference_urls

        for each in company_details:
            each = each[0]
            key = each.split(':')[0]
            value = each.split(':')[1]

            #Adding Table info from the soup
            output_record[key] = value   

        
        # print(output_record)
        final_output.append(output_record)

0it [00:00, ?it/s]

https://www.naics.com/company-profile-page/?co=11698


1it [00:00,  2.12it/s]

https://www.naics.com/company-profile-page/?co=10277


2it [00:00,  2.02it/s]

...

98it [00:47,  2.14it/s]

https://www.naics.com/company-profile-page/?co=8744


99it [00:48,  2.14it/s]

https://www.naics.com/company-profile-page/?co=5824


100it [00:48,  2.06it/s]


In [7]:
output_df = pd.DataFrame(final_output)

In [8]:
output_df.to_excel('company_details_f100.xlsx',index=False)